# Continuous-measuring Photocurrent

In [ ]:
#@markdown Step 1: File upload and input
filename = 'test_name' #@param {type:"string"}
amp_factor = 500 #@param {type:"number"}
avg_length = 1000 #@param {type:"number"}

#@markdown Step 2: Find peak
threshold = 0 #@param {type:"number"}
spacing = 1300 #@param {type:"number"}

#@markdown Step 3: Smoothing
window_length = 7 #@param {type:"number"}
polyorder = 2 #@param {type:"number"}

#@markdown Step 4: Mean calculation
lighton_time = 10 #@param {type:"number"}
lightoff_time = 35 #@param {type:"number"}
spacing_time = 5 #@param {type:"number"}

#@markdown Step 5: Figure
figure_title = 'new_test' #@param {type:"string"}
x_min = 0 #@param {type:"number"}
x_max = 45 #@param {type:"number"}
y_min = -10 #@param {type:"number"}
y_max = 50 #@param {type:"number"}


In [ ]:
#@title 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import matplotlib
from matplotlib.ticker import FormatStrFormatter
matplotlib.rcParams['agg.path.chunksize'] = 10000


def Baseline(df, amp, avg):
    shf = sum(df['Ampl'][0:avg]) / avg
    df['Ampl'] = [amp * (x - shf) for x in df['Ampl']]

    return df


def Findpeak(df, hh, dis):
    from scipy.signal import find_peaks
    peaks, _ = find_peaks(df['Ampl'], height=hh, distance=dis)
    np.diff(peaks)

    fpk_df = pd.DataFrame(columns=['Time', 'Ampl'])
    fpk_df['Time'] = df['Time'][peaks]
    fpk_df['Ampl'] = df['Ampl'][peaks]

    return fpk_df


def Savgol_filter(fpk_df, w_size, p_order):
    from scipy.signal import savgol_filter
    fpk_df["Ampl"] = savgol_filter(fpk_df["Ampl"], w_size, p_order)
    return fpk_df


def Mean(df, min, max):
    sel = []
    df.reset_index(drop=True, inplace=True)
    for i in range(len(df)):
        if (min <= df['Time'][i] <= max):
            sel.append(df['Ampl'][i])
    area = sum(sel)
    mean = area/len(sel)
    print("{}s to {}s: {} nA".format(min, max, round(mean, 3)))

    return round(mean, 3)


def Figure(df, input_path, title):
    fig, ax = plt.subplots(dpi=600, figsize=(8, 6))
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.tick_params(width=1.8)
    for axis in ['bottom', 'left']:
        ax.spines[axis].set_linewidth(1.8)
    font1 = {'family': 'arial', 'color':  'black',
             'weight': 'bold', 'size': 18}
    font2 = {'family': 'arial', 'color':  'black',
             'weight': 'bold', 'size': 26}
    plt.axvspan(lighton_time, lightoff_time, facecolor='green', alpha=0.1)
    plt.plot(df["Time"], df["Ampl"], c="grey", lw=0.1, alpha=0.9)
    plt.plot([0, 1250000], [0, 0], 'k:')
    plt.plot(fpk_df["Time"], fpk_df["Ampl"], c="red")

    plt.axis([x_min, x_max, y_min, y_max])
    plt.xlabel("Time (s)", fontdict=font1, labelpad=6)
    plt.ylabel("Photocurrent (nA)", fontdict=font1, labelpad=10)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.subplots_adjust(bottom=0.14, left=0.16)
    plt.title(title, fontdict=font2, pad=20)
    plt.savefig(input_path+".png")



import sys
import os
input_path = '/content/'+filename+'.csv'
df = pd.read_csv(input_path, skiprows=4)
input_path = os.path.splitext(input_path)[0]

df = Baseline(df, amp_factor, avg_length)
fpk_df = Findpeak(df, threshold, spacing)
fpk_df = Savgol_filter(fpk_df, window_length, polyorder)

cep = int((lightoff_time - lighton_time) / spacing_time)
for x in range(0, cep):
  Mean(fpk_df, lighton_time + spacing_time*x, lighton_time + spacing_time*(x+1))

Figure(df, input_path, figure_title)
print('Done! d(//-v-)b')
